In [ ]:
import os

import numpy as np
from himalaya.backend import set_backend
from matplotlib import pyplot as plt
from voxelwise_tutorials.utils import generate_leave_one_run_out

from compare_variance_residual.fmri.features import load_feature, load_brain_data

In [ ]:
data_dir = "../../data"
backend = set_backend("torch_cuda")

In [ ]:
subject = 7
modality = 'listening'
low_level_feature = "moten"

In [ ]:
alphas = np.logspace(-5, 15, 21)
number_of_delays = 4
n_targets_batch = 100
n_alphas_batch = 3
n_targets_batch_refit = 50

# Load data

In [ ]:
mapper_path = os.path.join("../../data", 'mappers', f"subject{subject:02}_mappers.hdf")
Y, n_samples_train, run_onsets, ev = load_brain_data(data_dir, subject, modality)
print(Y.shape)
print(n_samples_train)
print(Y.shape[0] - n_samples_train)

In [ ]:
X_low, n_samples_train = load_feature(data_dir, low_level_feature)
print(X_low.shape)
print(n_samples_train)
print(X_low.shape[0] - n_samples_train)

In [ ]:
X_high, n_samples_train = load_feature(data_dir, 'english1000')
print(X_high.shape)
print(n_samples_train)
print(X_high.shape[0] - n_samples_train)

# Run cross-feature model

In [ ]:
from himalaya.ridge import RidgeCV

# powspec 6 best

# moten
# 5 np.float32(0.0054762564)
# 6 np.float32(0.0055091255)
cross_model = RidgeCV(cv=5, alphas=alphas,
                      solver_params=dict(n_targets_batch=n_targets_batch, n_alphas_batch=n_alphas_batch,
                                         n_targets_batch_refit=n_targets_batch_refit))
cross_model.fit(X_low[:n_samples_train], X_high[:n_samples_train])
cross_model

In [ ]:
import seaborn as sns

sns.histplot(backend.to_numpy(cross_model.best_alphas_), log_scale=(True))
plt.yscale("log")

In [ ]:
score = cross_model.score(X_low[n_samples_train:], X_high[n_samples_train:])
score = backend.to_numpy(score)
score.mean()

In [ ]:
sns.histplot(score)

In [ ]:
for _cv in [7, 8, 9, 10, generate_leave_one_run_out(n_samples_train, run_onsets)]:
    cross_model = RidgeCV(cv=_cv, alphas=alphas,
                          solver_params=dict(n_targets_batch=n_targets_batch, n_alphas_batch=n_alphas_batch,
                                             n_targets_batch_refit=n_targets_batch_refit))
    cross_model.fit(X_low[:n_samples_train], X_high[:n_samples_train])
    score = cross_model.score(X_low[n_samples_train:], X_high[n_samples_train:])
    score = backend.to_numpy(score)
    print(_cv, score.mean())